In [1]:
import os
import math
import numpy
from typing import Any
from numpy.typing import NDArray
from numpy import (
    int64 as i64,
    uint8 as u8,
    float64 as f64,
)
import cv2
import ipywidgets # type: ignore
from fingerprint import (
    FE_CONFIG,
    FM_CONFIG,
    Fingerprint,
    Point,
    PointWithAngle,

    angles_abs_difference,
)
from features_extraction_utils import (
    RED,
    GREEN,
    float_slider,
    float_text,
    int_range_slider,
    int_slider,
    show,
    draw_directional_map_lines,
    draw_minutiae,
    draw_minutiae_with_angle,
    draw_minutiae_and_cylinder,
    draw_match_pairs
)
from utils import (
    DATASET_DIR_PATH,
    DATABASE_DIR_PATH,
    FINGERPRINTS_IMAGE_FILE_EXTENSION,
)


In [2]:
from features_extraction_utils import draw_singularities
from fingerprint import Minutiae, Singularities


finger_tag = "FVC2002/db1_b/101"
acquisition_tag = "2"
fingerprint_tag = f"{finger_tag}_{acquisition_tag}"
fingerprint_file_path = os.path.join(DATASET_DIR_PATH, fingerprint_tag) + FINGERPRINTS_IMAGE_FILE_EXTENSION
fingerprint_file_path = os.path.normpath(fingerprint_file_path)
print(fingerprint_file_path)

fingerprint: Fingerprint

@ipywidgets.interact(
    gradient_sobel_filter_length = int_slider(
        range = FE_CONFIG.gradient_sobel_filter_length,
        description = "gradient sobel filter length",
    ),
    gradient_module_block_length = int_slider(
        range = FE_CONFIG.gradient_module_block_length,
        description = "gradient module block length",
    ),
    segmentation_mask_threshold_scale = float_slider(
        range = FE_CONFIG.segmentation_mask_threshold_scale,
        description = "segmentation mask threshold scale",
    ),
    directional_map_block_length = int_slider(
        range = FE_CONFIG.directional_map_block_length,
        description = "directional map block lenght",
    ),
    directional_map_blur_filter_length = int_slider(
        range = FE_CONFIG.directional_map_blur_filter_length,
        description = "directional map blur length",
    ),
    local_ridge_block_rows = int_slider(
        range = FE_CONFIG.local_ridge_block_rows,
        description = "local ridge block rows",
    ),
    local_ridge_block_columns = int_slider(
        range = FE_CONFIG.local_ridge_block_columns,
        description = "local ridge block columns",
    ),
    gabor_filters_count = int_slider(
        range = FE_CONFIG.gabor_filters_count,
        description = "gabor filters count",
    ),
    gabor_filters_sigma = float_text(
        value = FE_CONFIG.gabor_filters_sigma,
        description = "gabor filters sigma",
    ),
    gabor_filters_gamma = float_text(
        value = FE_CONFIG.gabor_filters_gamma,
        description = "gabor filters gamma",
    ),
    binarization_threshold = int_slider(
        range = FE_CONFIG.binarization_threshold,
        description = "binarization threshold",
    ),
    singularities_min_distance_from_border = int_slider(
        range = FE_CONFIG.singularities_min_distance_from_border,
        description = "singularities min distance from border",
    ),
    minutiae_min_distance_from_border = int_slider(
        range = FE_CONFIG.minutiae_min_distance_from_border,
        description = "minutiae min distance from border",
    ),
    minutiae_followed_length = int_range_slider(
        bounds = FE_CONFIG.minutiae_followed_length,
        description = "minutiae followed length",
    ),
    mcc_gaussian_std = float_text(
        value = FE_CONFIG.mcc_gaussian_std,
        description = "mcc gaussian std",
    ),
    mcc_sigmoid_tau = float_text(
        value = FE_CONFIG.mcc_sigmoid_tau,
        description = "mcc sigmoid tau",
    ),
    mcc_sigmoid_mu = float_text(
        value = FE_CONFIG.mcc_sigmoid_mu,
        description = "mcc sigmoid mu",
    ),
) # type: ignore
def extract_features(
    gradient_sobel_filter_length: int,
    gradient_module_block_length: int,
    segmentation_mask_threshold_scale: float,
    directional_map_block_length: int,
    directional_map_blur_filter_length: int,
    local_ridge_block_rows: int,
    local_ridge_block_columns: int,
    gabor_filters_count: int,
    gabor_filters_sigma: float,
    gabor_filters_gamma: float,
    binarization_threshold: int,
    singularities_min_distance_from_border: int,
    minutiae_min_distance_from_border: int,
    minutiae_followed_length: tuple[int, int],
    mcc_gaussian_std: float,
    mcc_sigmoid_tau: float,
    mcc_sigmoid_mu: float,
) -> None:
    min_minutiae_followed_length, max_minutiae_followed_length = minutiae_followed_length

    global fingerprint
    fingerprint = Fingerprint(
        file_path = fingerprint_file_path,
        acquisition_tag = acquisition_tag,
        gradient_sobel_filter_length = gradient_sobel_filter_length,
        gradient_module_block_length = gradient_module_block_length,
        segmentation_mask_threshold_scale = segmentation_mask_threshold_scale,
        directional_map_block_length = directional_map_block_length,
        directional_map_blur_filter_length = directional_map_blur_filter_length,
        local_ridge_block_rows = local_ridge_block_rows,
        local_ridge_block_columns = local_ridge_block_columns,
        gabor_filters_count = gabor_filters_count,
        gabor_filters_sigma = gabor_filters_sigma,
        gabor_filters_gamma = gabor_filters_gamma,
        binarization_threshold = binarization_threshold,
        minutiae_min_distance_from_border = minutiae_min_distance_from_border,
        minutiae_followed_length_min = min_minutiae_followed_length,
        minutiae_followed_length_max = max_minutiae_followed_length,
        mcc_reference_cell_coordinates = None, # TODO(stefano): add sliders for cylinders controls
        mcc_gaussian_std = mcc_gaussian_std,
        mcc_sigmoid_tau = mcc_sigmoid_tau,
        mcc_sigmoid_mu = mcc_sigmoid_mu,
    )

    # show(
    #     (f"Raw fingerprint {fingerprint.raw_fingerprint.shape}", fingerprint.raw_fingerprint),
    #     (f"Normalized {fingerprint.normalized_fingerprint.shape}", fingerprint.normalized_fingerprint),
    #     (f"Normalized negative {fingerprint.normalized_negative_fingerprint.shape}", fingerprint.normalized_negative_fingerprint),
    # )

    # show(
    #     ("Gradient x", fingerprint.gradient_x),
    #     ("Gradient y", fingerprint.gradient_y),
    #     ("Gradient x**2", fingerprint.gradient_x2),
    #     ("Gradient y**2", fingerprint.gradient_y2),

    #     ("Gradient module", fingerprint.gradient_module),

    #     # ("Gradient x**2 filtered", fingerprint.gradient_x2_filtered),
    #     # ("Gradient y**2 filtered", fingerprint.gradient_y2_filtered),
    #     # ("Gradient x*y filtered", fingerprint.gradient_xy_filtered),

    #     # ("Gradient x**2 - y**2 filtered", fingerprint.gradient_x2_minus_y2_filtered),
    #     # ("Gradient 2x*y filtered", fingerprint.gradient_2xy_filtered),

    #     max_images_per_row = 5,
    # )

    # fingerprint_with_directional_lines = draw_directional_map_lines(
    #     fingerprint.raw_fingerprint,
    #     fingerprint.directional_map,
    #     fingerprint.segmentation_mask,
    #     directional_map_block_length,
    # )
    # show(
    #     ("Segmentation mask", fingerprint.segmentation_mask),
    #     ("Segmentation mask distance map", fingerprint.segmentation_mask_distance_map),
    #     ("Segmentation mask", cv2.merge((
    #         fingerprint.raw_fingerprint,
    #         fingerprint.raw_fingerprint,
    #         fingerprint.segmentation_mask
    #     ))),

    #     ("Directional Map", fingerprint_with_directional_lines),
    # )

    # fingerprint_with_highlighted_ridge_block = cv2.rectangle(
    #     cv2.cvtColor(fingerprint.raw_fingerprint, cv2.COLOR_GRAY2BGR),
    #     pt1 = (fingerprint.ridge_block_row_start, fingerprint.ridge_block_column_start),
    #     pt2 = (fingerprint.ridge_block_row_end, fingerprint.ridge_block_column_end),
    #     color = RED,
    #     thickness = 1,
    #     lineType = cv2.LINE_AA,
    # )
    # show(
    #     (f"Local ridge with average ridge frequency = {fingerprint.ridge_frequency}", fingerprint_with_highlighted_ridge_block),
    #     ("Enhanced fingeprint", fingerprint.enhanced_fingerprint),
    #     ("Binarized fingeprint", fingerprint.binarized_fingerprint),
    #     ("Thinned fingeprint", fingerprint.thinned_fingerprint),
    # )

    # fingerprint_with_gabor_filters: list[tuple[str, NDArray[u8]]] = []
    # gabor_filters: list[tuple[str, NDArray[f64]]] = []
    # for gabor_kernel, gabor_kernel_angle, fingerprint_with_gabor_filter in zip(
    #     fingerprint.gabor_filters,
    #     fingerprint.gabor_filters_angles,
    #     fingerprint.fingerprint_with_gabor_filters
    # ):
    #     angle_in_degrees = round(gabor_kernel_angle * 180 / numpy.pi, ndigits = 2)
    #     label = f"{angle_in_degrees}"
    #     gabor_filters.append((label, gabor_kernel))
    #     fingerprint_with_gabor_filters.append((label, fingerprint_with_gabor_filter))
    # show(
    #     *gabor_filters,
    #     *fingerprint_with_gabor_filters,
    #     max_images_per_row = gabor_filters_count
    # )

    singularities = Singularities(
        fingerprint.directional_map,
        directional_map_block_length,
        fingerprint.segmentation_mask_distance_map,
        singularities_min_distance_from_border,
    )
    minutiae = Minutiae(
        fingerprint.thinned_fingerprint,
        fingerprint.segmentation_mask_distance_map,
        minutiae_min_distance_from_border,
    )
    thinned_fingerprint_with_all_minutiae = draw_minutiae(
        fingerprint.thinned_fingerprint,
        minutiae.all,
    )
    thinned_fingerprint_with_filtered_minutiae = draw_minutiae(
        fingerprint.thinned_fingerprint,
        minutiae.filtered,
    )
    thinned_fingerprint_with_valid_minutiae = draw_minutiae_with_angle(
        fingerprint.thinned_fingerprint,
        fingerprint.acquisition.features.minutiae,
    )
    thinned_fingerprint_with_all_singularities = draw_singularities(
        fingerprint.thinned_fingerprint,
        singularities.all,
    )
    thinned_fingerprint_with_filtered_singularities = draw_singularities(
        fingerprint.thinned_fingerprint,
        singularities.filtered,
    )
    thinned_fingerprint_with_valid_singularities = draw_singularities(
        fingerprint.thinned_fingerprint,
        fingerprint.acquisition.features.singularities,
    )
    show(
        # ("All minutiae", thinned_fingerprint_with_all_minutiae),
        # ("Filtered minutiae", thinned_fingerprint_with_filtered_minutiae),
        # ("Valid minutiae with angles", thinned_fingerprint_with_valid_minutiae),
        ("All singularities", thinned_fingerprint_with_all_singularities),
        ("Filtered singularities", thinned_fingerprint_with_filtered_singularities),
        ("Valid singularities with angles", thinned_fingerprint_with_valid_singularities),

        max_images_per_row = 3,
    )


datasets\FVC2002\db1_b\101_2.tif


interactive(children=(IntSlider(value=1, description='gradient sobel filter length', layout=Layout(width='auto…

In [3]:
fingerprint_with_minutiae = draw_minutiae_with_angle(
    fingerprint.thinned_fingerprint,
    fingerprint.acquisition.features.minutiae,
    fingerprint.acquisition.features.minutiae_kinds,
)

@ipywidgets.interact(
    minutia_index = ipywidgets.IntSlider(
        min = 0,
        max = len(fingerprint.acquisition.features.minutiae) - 1,
        description = "minutia index",
        layout = ipywidgets.Layout(width = "auto"),
        style = {"description_width": "initial"}
    ),
) # type: ignore
def draw_cilinders(minutia_index: int) -> None:
    fingerprint_with_minutiae_and_cylinders = draw_minutiae_and_cylinder(
        fingerprint_with_minutiae.copy(),
        fingerprint.acquisition.features,
        minutia_index
    )
    show((f"cylinder {minutia_index}", fingerprint_with_minutiae_and_cylinders))


interactive(children=(IntSlider(value=0, description='minutia index', layout=Layout(width='auto'), max=56, sty…

In [ ]:
other_fingerprint_database_tag = "FVC2002/db1_b/101"
expected_identity_database_path = os.path.join(DATABASE_DIR_PATH, other_fingerprint_database_tag) + ".npz"

fingerprints: list[Fingerprint]
with numpy.load(expected_identity_database_path, allow_pickle = True) as expected_fingerprints_database:
    expected_fingerprints_database: NDArray[Any]
    fingerprints = expected_fingerprints_database["fingerprints"].tolist() # type: ignore

def matching_score_local_structures(
    acquisition_tag: int,
) -> tuple[float, list[tuple[str, NDArray[u8]]]]:
    other_fingerprint_tag = f"FVC2002/db1_b/101_{acquisition_tag}"
    other_fingerprint_file_path = os.path.join(DATASET_DIR_PATH, other_fingerprint_tag) + FINGERPRINTS_IMAGE_FILE_EXTENSION
    other_fingerprint_file_path = os.path.normpath(other_fingerprint_file_path)
    other_raw_fingerprint: NDArray[u8] = cv2.imread(other_fingerprint_file_path, flags = cv2.IMREAD_GRAYSCALE) # type: ignore

    other = fingerprints[acquisition_tag - 1]

    distances: NDArray[f64] = numpy.linalg.norm(
        fingerprint.acquisition.features.local_structures[:, numpy.newaxis,:] - other.acquisition.features.local_structures,
        axis = -1
    )
    distances /= numpy.linalg.norm(fingerprint.acquisition.features.local_structures, axis = 1)[:, numpy.newaxis] + numpy.linalg.norm(other.acquisition.features.local_structures, axis = 1)
    minutiae_matching_pairs: tuple[NDArray[i64], NDArray[i64]] = numpy.unravel_index(
        numpy.argpartition(
            distances,
            FM_CONFIG.local_structures_matching_minutiae_pair_count, None
        )[: FM_CONFIG.local_structures_matching_minutiae_pair_count],
        distances.shape
    ) # type: ignore
    matching_score = float(1 - numpy.mean(distances[minutiae_matching_pairs[0], minutiae_matching_pairs[1]]))

    fingerprint_with_minutiae = draw_minutiae_with_angle(fingerprint.raw_fingerprint, fingerprint.acquisition.features.minutiae, fingerprint.acquisition.features.minutiae_kinds)
    other_with_minutiae = draw_minutiae_with_angle(other_raw_fingerprint, other.acquisition.features.minutiae, other.acquisition.features.minutiae_kinds)

    fingerprint_columns, fingerprint_rows = fingerprint.raw_fingerprint
    if matching_score >= FM_CONFIG.matching_score_genuine_threshold.value:
        fingerprint_with_minutiae: NDArray[u8] = cv2.rectangle(
            fingerprint_with_minutiae,
            pt1 = (0, 0),
            pt2 = (fingerprint_columns - 1, fingerprint_rows - 1),
            color = GREEN,
            thickness = 1,
            lineType = cv2.LINE_AA,
        ) # type: ignore
    else:
        fingerprint_with_minutiae: NDArray[u8] = cv2.rectangle(
            fingerprint_with_minutiae,
            pt1 = (0, 0),
            pt2 = (fingerprint_columns - 1, fingerprint_rows - 1),
            color = RED,
            thickness = 1,
            lineType = cv2.LINE_AA,
        ) # type: ignore

    match_pairs_images: list[tuple[str, NDArray[u8]]] = []
    for matching_pair_index in range(len(minutiae_matching_pairs[0])):
        match_pairs = draw_match_pairs(
            fingerprint_with_minutiae.copy(),
            fingerprint.acquisition.features,
            other_with_minutiae.copy(),
            other.acquisition.features,
            minutiae_matching_pairs,
            matching_pair_index,
        )

        match_pair_name = f"match pair {matching_pair_index}"
        match_pairs_images.append((match_pair_name, match_pairs))

    return matching_score, match_pairs_images

matching_score, matching_pairs = matching_score_local_structures(
    acquisition_tag = 4,
)

def show_matching_pairs(matching_pair_index: int) -> None:
    show(matching_pairs[matching_pair_index])

show_matching_pairs_widget = ipywidgets.interactive(
    show_matching_pairs,
    matching_pair_index = ipywidgets.IntSlider(
        min = 0,
        max = len(matching_pairs) - 1,
        description = "matching pair index",
        layout = ipywidgets.Layout(width = "auto"),
        style = {"description_width": "initial"}
    ),
)

_ = display(matching_score, show_matching_pairs_widget)

In [ ]:
other_fingerprint_database_tag = "FVC2002/db1_b/101"
expected_identity_database_path = os.path.join(DATABASE_DIR_PATH, other_fingerprint_database_tag) + ".npz"

fingerprints: list[Fingerprint]
with numpy.load(expected_identity_database_path, allow_pickle = True) as expected_fingerprints_database:
    expected_fingerprints_database: NDArray[Any]
    fingerprints = expected_fingerprints_database["fingerprints"].tolist() # type: ignore

MINUTIAE_MATCHING_PIXELS_DISTANCE_THRESHOLD = 15
MINUTIAE_MATCHING_ANGLE_DEGREES_DISTANCE_THRESHOLD = 15

@ipywidgets.interact(
    acquisition_tag = ipywidgets.IntSlider(
        min = 1,
        max = len(fingerprints),
        description = "acquisition tag",
        layout = ipywidgets.Layout(width = "auto"),
        style = {"description_width": "initial"}
    ),
) # type: ignore
def matching_score_hough(acquisition_tag: int) -> float:
    def is_close(row_0: int, column_0: int, row_1: int, column_1: int) -> bool:
        return (column_0 - column_1) ** 2 + (row_0 - row_1) ** 2 <= (MINUTIAE_MATCHING_PIXELS_DISTANCE_THRESHOLD ** 2)

    def is_aligned(angle_0: float, angle_1: float) -> bool:
        return angles_abs_difference(angle_0, angle_1) <= MINUTIAE_MATCHING_ANGLE_DEGREES_DISTANCE_THRESHOLD

    other_fingerprint_tag = f"FVC2002/db1_b/101_{acquisition_tag}"
    other_fingerprint_file_path = os.path.join(DATASET_DIR_PATH, other_fingerprint_tag) + FINGERPRINTS_IMAGE_FILE_EXTENSION
    other_fingerprint_file_path = os.path.normpath(other_fingerprint_file_path)
    other_raw_fingerprint: NDArray[u8] = cv2.imread(other_fingerprint_file_path, flags = cv2.IMREAD_GRAYSCALE) # type: ignore

    global fingerprints
    other = fingerprints[acquisition_tag - 1]

    other_raw_fingerprint_rows: int
    other_raw_fingerprint_columns: int
    other_raw_fingerprint_rows, other_raw_fingerprint_columns = other_raw_fingerprint.shape

    max_rows = max(fingerprint_rows, other_raw_fingerprint_rows)
    max_columns = max(fingerprint_columns, other_raw_fingerprint_columns)

    translation_coordinates_accumulator_map: dict[PointWithAngle, int] = {}

    self_minutia_index = 0
    while self_minutia_index < len(minutiae):
        self_minutia = minutiae[self_minutia_index]
        self_minutia_index += 1

        other_minutia_index = 0
        while other_minutia_index < len(other.minutiae):
            other_minutia = other.minutiae[other_minutia_index]
            other_minutia_index += 1

            angles_difference = abs(self_minutia.angle - other_minutia.angle)
            rotation_angle = min(angles_difference, 360 - angles_difference)
            rotation_angle_cos = math.cos(rotation_angle)
            rotation_angle_sin = math.sin(rotation_angle)
            rotation_matrix: NDArray[f64] = numpy.array([
                [rotation_angle_cos, -rotation_angle_sin],
                [rotation_angle_sin, rotation_angle_cos],
            ])

            translation: NDArray[i64] = (
                Point(other_minutia.column, other_minutia.row)
                - rotation_matrix @ Point(self_minutia.column, self_minutia.row)).round().astype(i64)
            translation_column: i64
            translation_row: i64
            translation_column, translation_row = translation
            translation_point = PointWithAngle(
                column = int(translation_column),
                row = int(translation_row),
                angle = angles_difference
            )

            if translation_point in translation_coordinates_accumulator_map:
                translation_coordinates_accumulator_map[translation_point] += 1
            else:
                translation_coordinates_accumulator_map[translation_point] = 1

            # translation_columns_accumulator[translation_column] += 1
            # translation_angles_accumulator[int(round(rotation_angle))] += 1
            # self_normalized_minutiae[self_minutia_index] = Point(
            #     column = self_minutia.column + int(translation_column),
            #     row = self_minutia.row + int(translation_row)
            # )

    translation_coordinates_accumulator = numpy.array(list(translation_coordinates_accumulator_map.values()), dtype = i64)

    most_voted_translation_coordinates_index: i64 = translation_coordinates_accumulator.argmax()
    most_voted_translation_coordinates = list(translation_coordinates_accumulator_map.keys())[most_voted_translation_coordinates_index]

    # rotation_angle_cos = math.cos(most_voted_translation_angle)
    # rotation_angle_sin = math.sin(most_voted_translation_angle)
    # rotation_matrix: NDArray[f64] = numpy.array([
    #     [rotation_angle_cos, rotation_angle_sin],
    #     [-rotation_angle_sin, rotation_angle_cos],
    # ])

    minutiae_points = [
        Point(column, row) for column, row, _angle in minutiae
    ]
    self_aligned_minutiae_f64: NDArray[f64] = minutiae_points + numpy.array([most_voted_translation_coordinates.column, most_voted_translation_coordinates.row]) # type: ignore
    self_aligned_minutiae = [
        Point(int(round(column)), int(round(row))) for column, row in self_aligned_minutiae_f64
    ]

    h1, w1 = fingerprint_rows, fingerprint_columns
    h2, w2 = other_raw_fingerprint_rows, other_raw_fingerprint_columns
    res = numpy.full((max(h1,h2), w1+w2, 3), 255, u8)
    res[:h1,:w1] = draw_minutiae_with_angle(
        raw_fingerprint,
        minutiae,
        minutiae_kinds,
    )
    res[:h2,w1:w1+w2] = draw_minutiae_with_angle(
        other_raw_fingerprint,
        other.minutiae,
        other.minutiae_kinds,
    )

    matching_minutiae_count = 0
    for self_minutia, self_aligned_minutia in zip(minutiae, self_aligned_minutiae, strict = True):
        for other_minutia in other.minutiae:
            minutiae_are_close = is_close(
                self_aligned_minutia.row,
                self_aligned_minutia.column,
                other_minutia.row,
                other_minutia.column,
            )
            if minutiae_are_close:
                _ = cv2.line(
                    res,
                    pt1 = (self_minutia.column, self_minutia.row),
                    pt2 = (w1 + other_minutia.column, other_minutia.row),
                    color = GREEN,
                    thickness = 1,
                    lineType = cv2.LINE_AA
                )

                matching_minutiae_count += 1
                break

    other_fingerprint_with_self_aligned_minutiae = draw_minutiae(
        other_raw_fingerprint,
        self_aligned_minutiae,
        minutiae_kinds,
    )

    matching_score = matching_minutiae_count / max(len(self_aligned_minutiae), len(other.minutiae))

    if matching_score >= MATCHING_SCORE_GENUINE_THRESHOLD:
        res: NDArray[u8] = cv2.rectangle(
            res,
            pt1 = (0, 0),
            pt2 = (fingerprint_columns - 1, fingerprint_rows - 1),
            color = GREEN,
            thickness = 1,
            lineType = cv2.LINE_AA,
        ) # type: ignore
    else:
        res: NDArray[u8] = cv2.rectangle(
            res,
            pt1 = (0, 0),
            pt2 = (fingerprint_columns - 1, fingerprint_rows - 1),
            color = RED,
            thickness = 1,
            lineType = cv2.LINE_AA,
        ) # type: ignore

    show(
        ("both minutiae", res),
        ("self aligned minutiae", other_fingerprint_with_self_aligned_minutiae),
    )
    return matching_score

# matching_score = matching_score_hough(4)
# _ = display(matching_score)

In [ ]:
from typing import Any
from config import DATABASE_DIR_PATH, MATCHING_SCORE_GENUINE_THRESHOLD, LOCAL_STRUCTURES_MATCHING_MINUTIAE_PAIR_COUNT
from fingerprint import Fingerprint

other_fingerprint_database_tag = "FVC2002/db1_b/101"
expected_identity_database_path = os.path.join(DATABASE_DIR_PATH, other_fingerprint_database_tag) + ".npz"

fingerprints: list[Fingerprint]
with numpy.load(expected_identity_database_path, allow_pickle = True) as expected_fingerprints_database:
    expected_fingerprints_database: NDArray[Any]
    fingerprints = expected_fingerprints_database["fingerprints"].tolist() # type: ignore

MINUTIAE_MATCHING_PIXELS_DISTANCE_THRESHOLD = 15
MINUTIAE_MATCHING_ANGLE_DEGREES_DISTANCE_THRESHOLD = 15

@ipywidgets.interact(
    acquisition_tag = ipywidgets.IntSlider(
        min = 1,
        max = len(fingerprints),
        description = "acquisition tag",
        layout = ipywidgets.Layout(width = "auto"),
        style = {"description_width": "initial"}
    ),
) # type: ignore
def matching_score_hough(acquisition_tag: int) -> float:
    def is_close(row_0: int, column_0: int, row_1: int, column_1: int) -> bool:
        return (column_0 - column_1) ** 2 + (row_0 - row_1) ** 2 <= (MINUTIAE_MATCHING_PIXELS_DISTANCE_THRESHOLD ** 2)

    def is_aligned(angle_0: float, angle_1: float) -> bool:
        return angles_abs_difference(angle_0, angle_1) <= MINUTIAE_MATCHING_ANGLE_DEGREES_DISTANCE_THRESHOLD

    other_fingerprint_tag = f"FVC2002/db1_b/101_{acquisition_tag}"
    other_fingerprint_file_path = os.path.join(DATASET_DIR_PATH, other_fingerprint_tag) + FINGERPRINTS_IMAGE_FILE_EXTENSION
    other_fingerprint_file_path = os.path.normpath(other_fingerprint_file_path)
    other_raw_fingerprint: NDArray[u8] = cv2.imread(other_fingerprint_file_path, flags = cv2.IMREAD_GRAYSCALE) # type: ignore

    global fingerprints
    other = fingerprints[acquisition_tag - 1]

    other_raw_fingerprint_rows: int
    other_raw_fingerprint_columns: int
    other_raw_fingerprint_rows, other_raw_fingerprint_columns = other_raw_fingerprint.shape

    accumulator: dict[PointWithAngle, int] = {}

    for other_minutia in other.minutiae:
        for self_minutia in minutiae:
            angles_difference = abs(self_minutia.angle - other_minutia.angle)
            rotation_angle = min(angles_difference, 360 - angles_difference)
            rotation_angle_cos = math.cos(rotation_angle)
            rotation_angle_sin = math.sin(rotation_angle)
            rotation_matrix: NDArray[f64] = numpy.array([
                [rotation_angle_cos, -rotation_angle_sin],
                [rotation_angle_sin, rotation_angle_cos],
            ])

            translation: NDArray[i64] = (
                Point(other_minutia.column, other_minutia.row)
                - rotation_matrix @ Point(self_minutia.column, self_minutia.row)).round().astype(i64)
            translation_column: i64
            translation_row: i64
            translation_column, translation_row = translation
            translation_point = PointWithAngle(
                column = int(translation_column),
                row = int(translation_row),
                angle = angles_difference
            )
            if translation_point in accumulator:
                accumulator[translation_point] += 1
            else:
                accumulator[translation_point] = 1

    most_voted_translation_column: PointWithAngle = ... # type: ignore
    max_votes = 0
    for translation_point, votes in accumulator.items():
        if votes > max_votes:
            most_voted_translation = translation_point

    # rotation_angle_cos = math.cos(most_voted_translation_angle)
    # rotation_angle_sin = math.sin(most_voted_translation_angle)
    # rotation_matrix: NDArray[f64] = numpy.array([
    #     [rotation_angle_cos, -rotation_angle_sin],
    #     [rotation_angle_sin, rotation_angle_cos],
    # ])

    minutiae_points = [
        Point(column, row) for column, row, _angle in minutiae
    ]
    self_aligned_minutiae_f64: NDArray[f64] = minutiae_points + numpy.array([most_voted_translation.column, most_voted_translation.row]) # type: ignore
    self_aligned_minutiae = [
        Point(int(round(column)), int(round(row))) for column, row in self_aligned_minutiae_f64
    ]

    h1, w1 = fingerprint_rows, fingerprint_columns
    h2, w2 = other_raw_fingerprint_rows, other_raw_fingerprint_columns
    res = numpy.full((max(h1,h2), w1+w2, 3), 255, u8)
    res[:h1,:w1] = draw_minutiae_with_angle(
        raw_fingerprint,
        minutiae,
        minutiae_kinds,
    )
    res[:h2,w1:w1+w2] = draw_minutiae_with_angle(
        other_raw_fingerprint,
        other.minutiae,
        other.minutiae_kinds,
    )

    matching_minutiae_count = 0
    for self_minutia, self_aligned_minutia in zip(minutiae, self_aligned_minutiae, strict = True):
        for other_minutia in other.minutiae:
            minutiae_are_close = is_close(
                self_aligned_minutia.row,
                self_aligned_minutia.column,
                other_minutia.row,
                other_minutia.column,
            )
            if minutiae_are_close:
                _ = cv2.line(
                    res,
                    pt1 = (self_minutia.column, self_minutia.row),
                    pt2 = (w1 + other_minutia.column, other_minutia.row),
                    color = GREEN,
                    thickness = 1,
                    lineType = cv2.LINE_AA
                )

                matching_minutiae_count += 1
                break

    other_fingerprint_with_self_aligned_minutiae = draw_minutiae(
        other_raw_fingerprint,
        self_aligned_minutiae,
        minutiae_kinds,
    )

    matching_score = matching_minutiae_count / max(len(self_aligned_minutiae), len(other.minutiae))

    if matching_score >= MATCHING_SCORE_GENUINE_THRESHOLD:
        res: NDArray[u8] = cv2.rectangle(
            res,
            pt1 = (0, 0),
            pt2 = (fingerprint_columns - 1, fingerprint_rows - 1),
            color = GREEN,
            thickness = 1,
            lineType = cv2.LINE_AA,
        ) # type: ignore
    else:
        res: NDArray[u8] = cv2.rectangle(
            res,
            pt1 = (0, 0),
            pt2 = (fingerprint_columns - 1, fingerprint_rows - 1),
            color = RED,
            thickness = 1,
            lineType = cv2.LINE_AA,
        ) # type: ignore

    show(
        ("both minutiae", res),
        ("self aligned minutiae", other_fingerprint_with_self_aligned_minutiae),
    )
    return matching_score

# matching_score = matching_score_hough(4)
# _ = display(matching_score)